##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Overfit e underfit

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/overfit_and_underfit"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Veja em TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Execute no Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Veja Código Fonte no GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/overfit_and_underfit.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Baixar o notebook</a>
  </td>
</table>

Como sempre, o código neste exemplo usará a API `tf.keras`, sobre a qual você pode aprender mais no TensorFlow [guia Keras] (https://www.tensorflow.org/guide/keras).

Nos dois exemplos anteriores - [classificando texto] (https://www.tensorflow.org/tutorials/keras/text_classification_with_hub) e [prevendo a eficiência de combustível] (https://www.tensorflow.org/tutorials/keras/regression ) - vimos que a precisão do nosso modelo nos dados de validação atingia o pico após o treinamento por várias épocas e, então, estagnava ou começava a diminuir.

Em outras palavras, nosso modelo superajustava os dados de treinamento. Aprender a lidar com a super adaptação (overfitting) é importante. Embora muitas vezes seja possível obter alta precisão no *conjunto de treinamento*, o que realmente queremos é desenvolver modelos que generalizem bem para um *conjunto de testes* (ou dados que eles nunca viram antes).

O oposto de overfitting é *underfitting*. A falta de ajuste ocorre quando ainda há espaço para melhorias nos dados de teste. Isso pode acontecer por várias razões: se o modelo não for suficientemente poderoso, for excessivamente regularizado ou simplesmente não foi treinado por tempo suficiente. Isso significa que a rede não aprendeu os padrões relevantes nos dados de treinamento.

Se você treinar por muito tempo, o modelo começará a se ajustar demais e aprender padrões a partir dos dados de treinamento que não se generalizam nos dados de teste. Precisamos encontrar um equilíbrio. Entender como treinar para um número apropriado de épocas, como exploraremos a seguir, é uma habilidade útil.

Para evitar o ajuste excessivo, a melhor solução é usar dados de treinamento mais completos. O conjunto de dados deve cobrir toda a gama de entradas com as quais o modelo deve lidar. Dados adicionais podem ser úteis apenas se cobrir casos novos e interessantes.

Um modelo treinado em dados mais completos naturalmente se generalizará melhor. Quando isso não for mais possível, a próxima melhor solução é usar técnicas como regularização. Isso impõe restrições à quantidade e ao tipo de informação que seu modelo pode armazenar. Se uma rede puder apenas memorizar um pequeno número de padrões, o processo de otimização a forçará a se concentrar nos padrões mais importantes, com maior chance de generalizar bem.

Neste notebook, exploraremos várias técnicas comuns de regularização e as usaremos para aprimorar um modelo de classificação.

## Configuração

Antes de começar, é necessário importar alguns pacotes:

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import regularizers

print(tf.__version__)

In [0]:
!pip install git+https://github.com/tensorflow/docs

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

In [0]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile


In [0]:
logdir = pathlib.Path(tempfile.mkdtemp())/"tensorboard_logs"
shutil.rmtree(logdir, ignore_errors=True)


## O conjunto de dados Higgs

O objetivo deste tutorial não é fazer física de partículas; portanto, não menciona os detalhes do conjunto de dados. Ele contém 11&#x202F;000&#x202F;000 de exemplos, cada um com 28 recursos e um rótulo de classe binária.

In [0]:
gz = tf.keras.utils.get_file('HIGGS.csv.gz', 'https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz')

In [0]:
FEATURES = 28

A classe `tf.data.experimental.CsvDataset` pode ser usada para ler registros csv diretamente de um arquivo gzip sem etapa intermediária de descompressão.

In [0]:
ds = tf.data.experimental.CsvDataset(gz,[float(),]*(FEATURES+1), compression_type="GZIP")

Essa classe de leitor csv retorna uma lista de escalares para cada registro. A função a seguir repack essa lista de escalares em um par (feature_vector, label).

In [0]:
def pack_row(*row):
  label = row[0]
  features = tf.stack(row[1:],1)
  return features, label

TensorFlow is most efficient when operating on large batches of data.

Portanto, em vez de reembalar cada linha individualmente, crie um novo `conjunto de dados` que use lotes de 10000 exemplos, aplique a função `pack_row` a cada lote e divida os lotes novamente em registros individuais:

In [0]:
packed_ds = ds.batch(10000).map(pack_row).unbatch()

Dê uma olhada em alguns dos registros deste novo `packages_ds`.

Os recursos não são perfeitamente normalizados, mas isso é suficiente para este tutorial.

In [0]:
for features,label in packed_ds.batch(1000).take(1):
  print(features[0])
  plt.hist(features.numpy().flatten(), bins = 101)

Para manter este tutorial relativamente curto, use apenas as primeiras 1000 amostras para validação e as próximas 10.000 para treinamento:

In [0]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

Os métodos `Dataset.skip` e `Dataset.take` facilitam isso.

Ao mesmo tempo, use o método `Dataset.cache` para garantir que o carregador não precise reler os dados do arquivo em cada época:

In [0]:
validate_ds = packed_ds.take(N_VALIDATION).cache()
train_ds = packed_ds.skip(N_VALIDATION).take(N_TRAIN).cache()

In [0]:
train_ds

Esses conjuntos de dados retornam exemplos individuais. Use o método `.batch` para criar lotes de tamanho apropriado para treinamento. Antes do lote, lembre-se de `.shuffle` e `.repeat` o conjunto de treinamento.

In [0]:
validate_ds = validate_ds.batch(BATCH_SIZE)
train_ds = train_ds.shuffle(BUFFER_SIZE).repeat().batch(BATCH_SIZE)

## Demonstrar sobreajuste

A maneira mais simples de evitar o ajuste excessivo é começar com um modelo pequeno: um modelo com um pequeno número de parâmetros aprendíveis (que é determinado pelo número de camadas e pelo número de unidades por camada). No aprendizado profundo, o número de parâmetros aprendíveis em um modelo é frequentemente chamado de "capacidade" do modelo.

Intuitivamente, um modelo com mais parâmetros terá mais "capacidade de memorização" e, portanto, poderá aprender facilmente um mapeamento perfeito como um dicionário entre amostras de treinamento e seus alvos, um mapeamento sem nenhum poder de generalização, mas isso seria inútil ao fazer previsões em dados não vistos anteriormente.

Lembre-se sempre disso: os modelos de aprendizado profundo tendem a ser bons para se ajustar aos dados de treinamento, mas o verdadeiro desafio é a generalização, não é adequada.

Por outro lado, se a rede tiver recursos limitados de memorização, não poderá aprender o mapeamento tão facilmente. Para minimizar sua perda, ele terá que aprender representações compactadas que têm mais poder preditivo. Ao mesmo tempo, se você tornar seu modelo muito pequeno, ele terá dificuldade em se ajustar aos dados de treinamento. Há um equilíbrio entre "muita capacidade" e "capacidade insuficiente".

Infelizmente, não existe uma fórmula mágica para determinar o tamanho certo ou a arquitetura do seu modelo (em termos de número de camadas ou tamanho certo para cada camada). Você terá que experimentar usando uma série de arquiteturas diferentes.

Para encontrar um tamanho de modelo apropriado, é melhor começar com relativamente poucas camadas e parâmetros e começar a aumentar o tamanho das camadas ou adicionar novas camadas até ver retornos decrescentes na perda de validação.

Comece com um modelo simples usando apenas `layers.Dense` como linha de base, depois crie versões maiores e compare-as.

### Procedimento de Treinamento

Muitos modelos treinam melhor se você reduzir gradualmente a taxa de aprendizado durante o treinamento. Use `optimizers.schedules` para reduzir a taxa de aprendizado ao longo do tempo:

In [0]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

O código acima define um `schedule.InverseTimeDecay` para diminuir hiperbolicamente a taxa de aprendizado para 1/2 da taxa básica em 1000 épocas, 1/3 em 2000 épocas e assim por diante.

In [0]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (8,6))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')


Cada modelo neste tutorial usará a mesma configuração de treinamento. Portanto, configure-os de maneira reutilizável, começando com a lista de retornos de chamada.

O treinamento para este tutorial é executado por muitas épocas curtas. Para reduzir o ruído de registro, use o `tfdocs.EpochDots`, que simplesmente é um `.` para cada época e, e um conjunto completo de métricas a cada 100 épocas.

Em seguida, inclua `callbacks.EarlyStopping` para evitar tempos de treinamento longos e desnecessários. Note que este retorno de chamada está configurado para monitorar o `val_binary_crossentropy`, não o `val_loss`. Essa diferença será importante mais tarde.

Use `callbacks.TensorBoard` para gerar logs do TensorBoard para o treinamento.


In [0]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='val_binary_crossentropy', patience=200),
    tf.keras.callbacks.TensorBoard(logdir/name),
  ]

Da mesma forma, cada modelo usará as mesmas configurações `Model.compile` e` Model.fit`:

In [0]:
def compile_and_fit(model, name, optimizer=None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()
  model.compile(optimizer=optimizer,
                loss='binary_crossentropy',
                metrics=['accuracy', 'binary_crossentropy'])

  model.summary()

  history = model.fit(
    train_ds,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_data=validate_ds,
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### Modelo Minúsculo

Comece treinando um modelo linear:

In [0]:
tiny_model = tf.keras.Sequential([
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(1, activation='sigmoid')
])

In [0]:
size_histories = {}

In [0]:
size_histories['Tiny'] = compile_and_fit(tiny_model, 'sizes/Tiny')

Agora verifique como o modelo foi:

In [0]:
plotter = tfdocs.plots.HistoryPlotter(metric = 'binary_crossentropy', smoothing_std=10)
plotter.plot(size_histories)
plt.ylim([0.5, 0.7])

### Pequeno Modelo

Para ver se você consegue superar o desempenho do modelo pequeno, treine progressivamente alguns modelos maiores.

Tente duas camadas ocultas com 16 unidades cada:

In [0]:
small_model = tf.keras.Sequential([
    # `input_shape` is only required here so that `.summary` works.
    layers.Dense(16, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(16, activation='elu'),
    layers.Dense(1, activation='sigmoid')
])

In [0]:
size_histories['Small'] = compile_and_fit(small_model, 'sizes/Small')

### Modelo Médio

Agora tente 3 camadas ocultas com 64 unidades cada:

In [0]:
medium_model = tf.keras.Sequential([
    layers.Dense(64, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(64, activation='elu'),
    layers.Dense(64, activation='elu'),
    layers.Dense(1, activation='sigmoid')
])

E treine o modelo usando os mesmos dados:

In [0]:
size_histories['Medium']  = compile_and_fit(medium_model, "sizes/Medium")

### Modelo grande

Como exercício, você pode criar um modelo ainda maior e ver com que rapidez ele começa a se ajustar. Em seguida, vamos adicionar a esse benchmark uma rede que tem muito mais capacidade, muito mais do que o problema justificaria:

In [0]:
large_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(512, activation='elu'),
    layers.Dense(1, activation='sigmoid')
])

E, novamente, treine o modelo usando os mesmos dados:

In [0]:
size_histories['large'] = compile_and_fit(large_model, "sizes/large")

### Traçar as Perdas de Treinamento e Validação

As linhas sólidas mostram a perda de treinamento e as linhas tracejadas mostram a perda de validação (lembre-se: uma perda de validação menor indica um modelo melhor).

Embora a construção de um modelo maior ofereça mais poder, se esse poder não for restringido de alguma forma, pode facilmente se sobrepor ao conjunto de treinamento.

Neste exemplo, normalmente, apenas o modelo `" Tiny "` consegue evitar o excesso de ajuste, e cada um dos modelos maiores superajusta os dados mais rapidamente. Isso se torna tão severo para o modelo `` grande '' que você precisa mudar o gráfico para uma escala de log para realmente ver o que está acontecendo.

Isso é aparente se você plotar e comparar as métricas de validação com as métricas de treinamento.

* É normal que haja uma pequena diferença.
* Se as duas métricas estiverem se movendo na mesma direção, tudo estará bem.
* Se a métrica de validação começar a estagnar enquanto a métrica de treinamento continuar a melhorar, você provavelmente estará com excesso de ajustes.
* Se a métrica de validação estiver indo na direção errada, o modelo estará claramente sobregravado.

In [0]:
plotter.plot(size_histories)
a = plt.xscale('log')
plt.xlim([5, max(plt.xlim())])
plt.ylim([0.5, 0.7])
plt.xlabel("Epochs [Log Scale]")

Nota: Todas as execuções de treinamento acima usaram o `callbacks.EarlyStopping` para finalizar o treinamento, uma vez que ficou claro que o modelo não estava progredindo.

### Visualizar no TensorBoard

Todos esses modelos gravaram logs do TensorBoard durante o treinamento.

Para abrir um visualizador TensorBoard incorporado em um notebook, copie o seguinte em uma célula de código:

```
%tensorboard --logdir {logdir}/sizes
```

Você pode visualizar os [resultados de uma execução anterior] (https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97) deste notebook em [TensorBoard.dev] (https://tensorboard.dev/).

O TensorBoard.dev é uma experiência gerenciada para hospedar, rastrear e compartilhar experimentos de ML com todos.

Também está incluído em um `<iframe>` por conveniência:

In [0]:
display.IFrame(
    src="https://tensorboard.dev/experiment/vW7jmmF9TmKmy3rbheMQpw/#scalars&_smoothingWeight=0.97",
    width="100%", height="800px")

Se você deseja compartilhar os resultados do TensorBoard, pode fazer o upload dos logs para [TensorBoard.dev] (https://tensorboard.dev/), copiando o seguinte em uma célula de código.

Nota: Esta etapa requer uma conta Google.

```
!tensorboard dev upload --logdir  {logdir}/sizes
```

Cuidado: Este comando não termina. Ele foi projetado para carregar continuamente os resultados de experimentos de longa duração. Após o upload dos dados, você precisa interrompê-los usando a opção "interromper a execução" na sua ferramenta de notebook.

## Estratégias para previnir o overfitting

Antes de entrar no conteúdo desta seção, copie os logs de treinamento do modelo ``Minúsculo`` acima, para usar como linha de base para comparação.

In [0]:
shutil.rmtree(logdir/'regularizers/Tiny', ignore_errors=True)
shutil.copytree(logdir/'sizes/Tiny', logdir/'regularizers/Tiny')

In [0]:
regularizer_histories = {}
regularizer_histories['Tiny'] = size_histories['Tiny']

### Adicione Regularização do Peso



Você pode estar familiarizado com o princípio Navalha de Occam: dadas duas explicações para algo, a explicação mais provável de ser correta é a "mais simples", a que apresenta o menor número de suposições. Isso também se aplica aos modelos aprendidos pelas redes neurais: dados alguns dados de treinamento e uma arquitetura de rede, existem vários conjuntos de valores de pesos (vários modelos) que poderiam explicar os dados, e modelos mais simples têm menos probabilidade de se superestimar do que os complexos.

Um "modelo simples" neste contexto é um modelo em que a distribuição dos valores dos parâmetros tem menos entropia (ou um modelo com menos parâmetros, como vimos na seção acima). Assim, uma maneira comum de mitigar o excesso de ajuste é restringir a complexidade de uma rede, forçando seus pesos apenas a aceitar valores pequenos, o que torna a distribuição dos valores de peso mais "regular". Isso é chamado de "regularização de peso" e é feito adicionando à função de perda da rede um custo associado a ter grandes pesos. Esse custo vem em dois sabores:

* [Regularização L1] (https://developers.google.com/machine-learning/glossary/#L1_regularization), em que o custo adicionado é proporcional ao valor absoluto dos coeficientes de pesos (ou seja, ao que é chamado de "norma L1 "dos pesos).

* [Regularização de L2] (https://developers.google.com/machine-learning/glossary/#L2_regularization), em que o custo adicionado é proporcional ao quadrado do valor dos coeficientes de pesos (ou seja, ao que é chamado de quadrado) "Norma L2" dos pesos). A regularização de L2 também é chamada de decaimento de peso no contexto de redes neurais. Não deixe que o nome diferente o confunda: a redução de peso é matematicamente a mesma que a regularização de L2.

A regularização L1 empurra pesos para exatamente zero, incentivando um modelo esparso. A regularização de L2 penalizará os parâmetros de pesos sem torná-los escassos, pois a penalidade é zero para pesos pequenos. uma razão pela qual L2 é mais comum.

Em `tf.keras`, a regularização de peso é adicionada passando instâncias de regularizador de peso para as camadas como argumentos de palavras-chave. Vamos adicionar a regularização de peso L2 agora.

In [0]:
l2_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001),
                 input_shape=(FEATURES,)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(512, activation='elu',
                 kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(1, activation='sigmoid')
])

regularizer_histories['l2'] = compile_and_fit(l2_model, "regularizers/l2")

`l2 (0,001)` significa que todo coeficiente na matriz de peso da camada adicionará `0,001 * weight_coefficient_value * 2` à **perda total** da rede.

É por isso que estamos monitorando o `binary_crossentropy` diretamente. Porque ele não possui esse componente de regularização misturado.

Portanto, o mesmo modelo "Grande" com uma penalidade de regularização `L2` tem um desempenho muito melhor:


In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Como você pode ver, o modelo regularizado `" L2 "` agora é muito mais competitivo com o modelo ``Minúsculo ``. Este modelo `" L2 "` também é muito mais resistente à super adaptação do que o modelo `` Large`` no qual foi baseado, apesar de ter o mesmo número de parâmetros.

#### Mais informações

Há duas coisas importantes a serem observadas sobre esse tipo de regularização.

**Primeiro:** se você estiver escrevendo seu próprio ciclo de treinamento, precisará solicitar ao modelo as perdas de regularização.

In [0]:
result = l2_model(features)
regularization_loss = tf.add_n(l2_model.losses)

**Segundo:** Essa implementação funciona adicionando as penalidades de peso à perda do modelo e aplicando um procedimento de otimização padrão depois disso.

Há uma segunda abordagem que executa o otimizador apenas na perda bruta e, ao aplicar a etapa calculada, o otimizador também aplica alguma redução de peso. Este "Decapled Weight Decay" é visto em otimizadores como `optimizers.FTRL` e `optimizers.AdamW`.

### Adicionar dropout

O abandono é uma das técnicas de regularização mais eficazes e mais usadas para redes neurais, desenvolvida por Hinton e seus alunos da Universidade de Toronto.

A explicação intuitiva para o abandono é que, como nós individuais na rede não podem confiar na saída dos outros, cada nó deve gerar recursos úteis por si só.

A desistência, aplicada a uma camada, consiste em "desistir" aleatoriamente (isto é, definido como zero) uma série de recursos de saída da camada durante o treinamento. Digamos que uma determinada camada retornaria normalmente um vetor [0,2, 0,5, 1,3, 0,8, 1,1] para uma determinada amostra de entrada durante o treinamento; depois de aplicar a desistência, esse vetor terá algumas entradas zero distribuídas aleatoriamente, por exemplo [0, 0,5,
1,3, 0, 1,1].

A "taxa de desistência" é a fração dos recursos que estão sendo zerados; geralmente é definido entre 0,2 e 0,5. No momento do teste, nenhuma unidade é eliminada e, em vez disso, os valores de saída da camada são reduzidos por um fator igual à taxa de desistência, de modo a equilibrar o fato de que mais unidades estão ativas do que no tempo de treinamento.

No `tf.keras`, você pode introduzir dropout em uma rede através da camada Dropout, que é aplicada à saída da camada logo antes.

Vamos adicionar duas camadas do Dropout em nossa rede para ver o desempenho delas na redução do overfitting:

In [0]:
dropout_model = tf.keras.Sequential([
    layers.Dense(512, activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

regularizer_histories['dropout'] = compile_and_fit(dropout_model, "regularizers/dropout")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Está claro neste gráfico que ambas as abordagens de regularização melhoram o comportamento do modelo `` Grande``. Mas isso ainda não supera nem a linha de base ``Tiny``.

Em seguida, tente os dois juntos e veja se isso funciona melhor.

### Combinando L2 + dropout

In [0]:
combined_model = tf.keras.Sequential([
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu', input_shape=(FEATURES,)),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(512, kernel_regularizer=regularizers.l2(0.0001),
                 activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

regularizer_histories['combined'] = compile_and_fit(combined_model, "regularizers/combined")

In [0]:
plotter.plot(regularizer_histories)
plt.ylim([0.5, 0.7])

Este modelo com a regularização ``"Combinada"`` é obviamente o melhor até agora.

### Vizualizar no TensorBoard

Esses modelos também registraram logs do TensorBoard.

Para abrir um visualizador de tensorboard incorporado em um notebook, copie o seguinte em uma célula de código:

```
%tensorboard --logdir {logdir}/regularizers
```

Você pode visualizar os [resultados de uma execução anterior] (https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97) deste notebook em [TensorDoard.dev] (https://tensorboard.dev/).

Também está incluído em um `<iframe>` por conveniência:

In [0]:
display.IFrame(
    src="https://tensorboard.dev/experiment/fGInKDo8TXes1z7HQku9mw/#scalars&_smoothingWeight=0.97",
    width = "100%",
    height="800px")


Este foi enviado com:

```
!tensorboard dev upload --logdir  {logdir}/regularizers
```

## Conclusões

Para recapitular: aqui estão as maneiras mais comuns de evitar o ajuste excessivo nas redes neurais:

* Obtenha mais dados de treinamento.
* Reduza a capacidade da rede.
* Adicione regularização de peso.
* Adicione desistência.

Duas abordagens importantes não abordadas neste guia são:

* aumento de dados
* normalização de lote

Lembre-se de que cada método pode ajudar por si só, mas geralmente combiná-los pode ser ainda mais eficaz.